In [1]:
from IPython.display import display
from ipywidgets import FloatProgress

import sys
sys.path.append('../src')

import torch
import neuralnetworkclassifier as nnc
import dataset_manipulations as dm
import perturb as per
import numpy as np
import mlutils as ml
import pickle

import matplotlib.pyplot as plt

In [2]:
Xtrain, Ttrain = dm.load_cifar_10('./cifar-10-batches-py/data_batch_*')
Xtest, Ttest = dm.load_cifar_10('./cifar-10-batches-py/test_batch')

In [3]:
noise_Xtrain = dm.apply_manipulations(Xtrain, per_func=lambda x: per.add_image_noise(x, variance=0.05))
noise_Xtest = dm.apply_manipulations(Xtest, per_func=lambda x: per.add_image_noise(x, variance=0.05))

In [4]:
lessnoise_Xtrain = dm.apply_manipulations(Xtrain, per_func=lambda x: per.add_image_noise(x, variance=0.025))
lessnoise_Xtest = dm.apply_manipulations(Xtest, per_func=lambda x: per.add_image_noise(x, variance=0.0025)) 

In [14]:
import imp
imp.reload(nnc)
imp.reload(per)

<module 'perturb' from '../src/perturb.py'>

In [15]:
nnet = per.train_cifar(Xtrain, Ttrain, verbose=True, random_seed=123)

Epoch 2 error 0.68250
Epoch 4 error 0.47265
Epoch 6 error 0.28336
Epoch 8 error 0.22093
Epoch 10 error 0.21511
Epoch 12 error 0.14268
Epoch 14 error 0.10318
Epoch 16 error 0.15704
Epoch 18 error 0.04334
Epoch 20 error 0.03581


In [16]:
nnet

NeuralNetwork_Convolutional(
                            n_channels_in_image=3,
                            image_size=32,
                            n_units_in_conv_layers=[64, 64, 128, 128, 256, 256, 512, 512],
                            kernels_size_and_stride=[(3, 1, 1), (3, 1, 1), (3, 1, 1), (3, 1, 1), (3, 1, 1), (3, 1, 1), (3, 1, 1), (3, 1, 1)],
                            max_pooling_kernels_and_stride=[(), (2, 2), (), (2, 2), (), (2, 2), (), (2, 2)],
                            n_units_in_fc_hidden_layers=[],
                            classes=[0 1 2 3 4 5 6 7 8 9],
                            use_gpu=True)
Sequential(
  (conv_0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (norm_0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (output_0): ReLU()
  (conv_1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (norm_1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  

In [17]:
clean_pct = ml.percent_correct(Ttest, ml.batched_use(nnet, Xtest))
noise_pct = ml.percent_correct(Ttest, ml.batched_use(nnet, noise_Xtest))
lessnoise_pct = ml.percent_correct(Ttest, ml.batched_use(nnet, lessnoise_Xtest))
(clean_pct, lessnoise_pct, noise_pct)

(86.45, 86.42, 67.63)

In [20]:
nnet.nnet[-1].in_features

2048

In [21]:
other_network = nnet.transfer_learn_setup([torch.nn.Linear(2048, 256), torch.nn.ReLU(), torch.nn.Linear(256, 10)])
other_network

Sequential(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU()
  (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (5): ReLU()
  (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (7): Dropout(p=0.2, inplace=False)
  (8): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (9): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (10): ReLU()
  (11): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (12): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (13): ReLU()
  (14): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (15): Dropout(p=0.2, inplace=False)
  (16): Conv2d(128, 256, kernel_size=(3, 3),

In [22]:
nnet.nnet = other_network

In [23]:
nnet.train(noise_Xtrain, Ttrain, n_epochs=10, batch_size=200,
           optim='Adam', learning_rate=0.0005, verbose=True)

Epoch 1 error 0.23455
Epoch 2 error 0.19167
Epoch 3 error 0.16644
Epoch 4 error 0.14729
Epoch 5 error 0.13183
Epoch 6 error 0.12282
Epoch 7 error 0.11871
Epoch 8 error 0.12230
Epoch 9 error 0.12084
Epoch 10 error 0.10975


In [24]:
clean_pct = ml.percent_correct(Ttest, ml.batched_use(nnet, Xtest))
noise_pct = ml.percent_correct(Ttest, ml.batched_use(nnet, noise_Xtest))
lessnoise_pct = ml.percent_correct(Ttest, ml.batched_use(nnet, lessnoise_Xtest))
(clean_pct, noise_pct, lessnoise_pct)

(83.48, 82.80999999999999, 83.55)